In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTENC
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score, confusion_matrix, roc_curve
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import average_precision_score

In [2]:
import plotly.graph_objects as go
import plotly.express as px
from sklearn.metrics import average_precision_score

In [3]:
data=pd.read_csv(r'C:/Users/HCJ/Desktop/2021_Summer/Finance_data/bank.csv',index_col=0)
print(data.shape)

(119535, 93)


In [4]:
train_data, test_data = train_test_split(data,test_size=0.2, random_state=100)
print(train_data.shape)
print(test_data.shape)

(95628, 93)
(23907, 93)


In [5]:
X_train = train_data.iloc[:,1:]
y_train = train_data['TARGET']
X_test = test_data.iloc[:,1:]
y_test = test_data['TARGET']

In [6]:
X_train.head()

,J_F1_LON_RK_3M_CNT_RE,J_F1_OTHER_LON_6M_AVBL_AM_RE,J_F1_OTHER_LON_MAX_3M_AM_RE,J_SHINBO_GIBO_AM_RE,J_TOT_LON_CNT_RE,J_TOT_LON_AGG_AM_RE,J_TOT_LON_MAX_AM_RE,J_DPS_CT_BAL_AM_RE,J_TOT_DPS_1M_AVBL_AM_RE,J_TOT_DPS_6M_AVBL_AM_RE,...,P_BR_CNTC_1M_CNT_RE,P_BR_CNTC_6M_CNT_RE,P_ATM_CNTC_6M_CNT_RE,P_INT_CNTC_6M_CNT_RE,P_TOT_LON_CNT_RE,P_TOT_LON_AGG_AM_RE,P_TOT_TS_6M_CNT_RE,P_TOT_TS_6M_AM_RE,P_OB_TS_6M_CNT_RE,P_OB_TS_6M_AM_RE
114140,1,2,3,1,3,3,3,2,2,3,...,1,1,1,1,1,1,1,1,1,1
17613,1,1,2,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
30297,3,3,3,1,1,1,1,1,1,1,...,1,1,3,2,3,3,3,3,3,3
54831,2,3,3,1,1,1,1,2,2,2,...,1,1,1,2,1,1,1,1,1,1
117127,3,3,4,2,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [7]:
y_train.head()

114140    0
17613     0
30297     0
54831     0
117127    0
Name: TARGET, dtype: int64

In [8]:
smotenc = SMOTENC(categorical_features = [42,43,44,63,64,65,66,67])
X_train_samp, y_train_samp = smotenc.fit_resample(X_train,y_train)

In [9]:
def get_eval(y_test,pred=None):
    confusion=confusion_matrix(y_test,pred)
    accuracy=accuracy_score(y_test,pred)
    precision=precision_score(y_test,pred)
    recall=recall_score(y_test, pred)
    f1 = f1_score(y_test,pred)
    precision2=precision_score(y_test,pred,pos_label=0)
    recall2=recall_score(y_test,pred,pos_label=0)
    f1_2=f1_score(y_test,pred,pos_label=0)
    print("confusion matrix")
    print(confusion)
    print('accuracy : {0:.4f} , precision : {1:.4f} , recall : {2:.4f}, F1 score : {3:.4f} \n precision(0) : {4:.4f} , recall(0) : {5:.4f}, f1(0) : {6:.4f}'.format(accuracy,precision, recall,f1,precision2,recall2,f1_2))
    
def precision_recall_curve_plot(y_test, pred_proba_cl):
    precisions, recalls, thresholds = precision_recall_curve(y_test, pred_proba_cl)
    plt.figure(figsize=(8,6))
    threshold_boundary = thresholds.shape[0]
    plt.plot(thresholds, precisions[0:threshold_boundary], linestyle='--', label = 'precision')
    plt.plot(thresholds, recalls[0:threshold_boundary], label = 'recall')
    start, end = plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1),2))
    plt.xlabel('Threshold value')
    plt.ylabel('Precision and Recall value')
    plt.legend()
    plt.grid()
    plt.show()

def roc_curve_plot(y_test, pred_proba_c1):
    fprs, tprs, thresholds = roc_curve(y_test, pred_proba_c1)
    plt.plot(fprs, tprs, label = 'ROC')
    plt.plot([0,1],[0,1],'k--',label='Random')
    start, end = plt.xlim()
    plt.xticks(np.round(np.arange(start,end,0.1),2))
    plt.xlim(0,1);plt.ylim(0,1)
    plt.xlabel('FPR(1-Sensitivity)');plt.ylabel('TPR(Recall)')
    plt.legend()

#randomforest model - hyperparameter tuning using grid search
params = { 'n_estimators': [100, 150, 200],
            'max_depth' : [10, 20, 30],
            'min_samples_leaf' : [8, 12, 18],
            'min_samples_split' : [2, 5, 10]
}

# RandomForestClassifier 객체 생성 후 GridSearchCV 수행
rf_clf = RandomForestClassifier(random_state = 0, n_jobs = -1)
grid_cv = GridSearchCV(rf_clf, param_grid = params, cv=3, n_jobs = -1, verbose=2)
grid_cv.fit(X_train_samp, y_train_samp)

print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))

In [10]:
# test 파라미터
rf_clf1 =RandomForestClassifier(max_depth = 10,
                                min_samples_leaf = 30,
                                min_samples_split = 20,
                                n_estimators = 100)

rf_clf1.fit(X_train_samp, y_train_samp)
pred = rf_clf1.predict(X_test)
get_eval(y_test,pred)

confusion matrix
[[22204  1547]
 [   30   126]]
accuracy : 0.9340 , precision : 0.0753 , recall : 0.8077, F1 score : 0.1378 
 precision(0) : 0.9987 , recall(0) : 0.9349, f1(0) : 0.9657


In [11]:
# best 파라미터
rf_clf2 =RandomForestClassifier(max_depth = 30,
                                min_samples_leaf = 8,
                                min_samples_split = 2,
                                n_estimators = 150)

rf_clf2.fit(X_train_samp, y_train_samp)
pred2 = rf_clf2.predict(X_test)
get_eval(y_test,pred2)

confusion matrix
[[23622   129]
 [   62    94]]
accuracy : 0.9920 , precision : 0.4215 , recall : 0.6026, F1 score : 0.4960 
 precision(0) : 0.9974 , recall(0) : 0.9946, f1(0) : 0.9960


In [12]:
# 샘플링 미적용
rf_clf3 =RandomForestClassifier(max_depth = 30,
                                min_samples_leaf = 8,
                                min_samples_split = 2,
                                n_estimators = 200)

rf_clf3.fit(X_train, y_train)
pred3 = rf_clf3.predict(X_test)
get_eval(y_test,pred3)

confusion matrix
[[23751     0]
 [  156     0]]
accuracy : 0.9935 , precision : 0.0000 , recall : 0.0000, F1 score : 0.0000 
 precision(0) : 0.9935 , recall(0) : 1.0000, f1(0) : 0.9967


C:\Users\HCJ\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
# 디폴트
rf_clf4 = RandomForestClassifier()

rf_clf4.fit(X_train_samp, y_train_samp)
pred4 = rf_clf4.predict(X_test)
get_eval(y_test,pred4)

confusion matrix
[[23726    25]
 [   60    96]]
accuracy : 0.9964 , precision : 0.7934 , recall : 0.6154, F1 score : 0.6931 
 precision(0) : 0.9975 , recall(0) : 0.9989, f1(0) : 0.9982


Precision Recall Plot 
Precision Recall Plot (이하 PR 그래프)의 경우도 ROC 와 유사한데, 주로 데이타 라벨의 분포가 심하게 불균등 할때 사용한데, 예를 들어 이상 거래 검출 시나리오의 경우 정상 거래의 비율이 비정상 거래에 비해서 압도적으로 많기 때문에 (98%, 2%) 이런 경우에는 ROC 그래프보다 PR 그래프가 분석에 더 유리하다.
https://bcho.tistory.com/1206